# data collection

Collect lots of things from lots of sources.




## basic http tools

Copying files from many sources.


### copy list of links to json

https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=400

Crime reports?


#### the code



In [ ]:
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crimes');

function scrapeAlert(ID) {
    if(fs.existsSync(path.join(project, 'maricopa-alert-' + ID + '.json'))) {
        return;
    }
    return client
        .url('https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=' + ID)
        .getAllXPath({
            time: '//*[@class = "single"]//*[@class = "date"]//text()',
            title: '//*[contains(@class, "single")]//h3//text()',
            content: '//*[@class = "single"]//*[@class = "content"]//text()'
        })
        .then(r => {
            fs.writeFileSync(path.join(project, 'maricopa-alert-' + ID + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
}
module.exports = scrapeAlert;

if(typeof $$ !== 'undefined') {
    $$.async();
    var IDs = Array.from(Array(500).keys());
    multiCrawl(IDs, 'crime reports')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


### search a lot of engines




#### the code 

meta search all?


In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var multiCrawl = importer.import('multi crawl');

// http://www.exploratorium.edu/files/ronh/research/index.html
/*

 Exploratorium	
 Search
 Google	
 Search
 alltheweb	
 Search
 Teoma	
 Search
 AltaVista	
 Search
 Wisenut	
 Search
 HotBot	
 Search
 lii.org	
 Search
 Northern Light	
 Search
 Lycos	
 Search
 Scirus	
 
 Meta:
  Ask Jeeves	
 Search
 MetaCrawler	
 Search
 Dogpile	
 Search
 SavvySearch	
 
 
 */
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/searches');

function searchAll(query = 'search engine') {
    var engines = [
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'http://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://www.yandex.com/search/?text=' + query,
        'https://archive.org/search.php?query=' + query,
    ];
    
    // TODO: save results
    return multiCrawl(engines, 'search results json')
        .then(r => {
            const time = new Date();
            fs.writeFileSync(path.join(project, query.replace(/[^a-z0-9]/ig, '_')
                                       + '-' + time.getFullYear()
                                       + '-' + (time.getMonth() + 1)
                                       + '-' + time.getDate()
                                       + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
}
module.exports = searchAll;

if(typeof $$ !== 'undefined') {
    $$.async();
    searchAll()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


#### schedule search all?

TODO: convert this to a pattern.



In [ ]:
var google = require('googleapis');
var importer = require('../Core');
var {
    getOauthClient,
    ISODateString,
    createNewEvent,
} = importer.import([
    'convert date iso',
    'create new calendar event',
    'import google calendar api',
]);

var options = {
    calendarId: 'aws'
}

function scheduleSearch(search) {
    const parameters = {
        query: search || 'search engines'
    }
    const newDate = new Date();
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => createNewEvent('meta search all', JSON.stringify(parameters, null, 4), options))
}
module.exports = scheduleSearch;


#### tell joke?



In [ ]:
var util = require('bluebird');
var request = util.promisify(require('request'));
var importer = require('../Core')

var jokes;
function getJoke() {
    // TODO: collect jokes instead
    return (typeof jokes === 'undefined'
        ? request('http://www.ducksters.com/jokes/silly.php')
        .then(res => importer.regexToArray(/^.*?Q:.*$|^.*?A:.*$/igm, res.body))
        .then(r => {
            r = r.reduce((arr, j, i) => {
                if(i % 2 === 1) {
                    arr.push([
                        r[i-1].replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, ''),
                        j.replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, '')
                    ]);
                }
                return arr;
            }, []);
            console.log(r);
            jokes = r;
            return r;
        })
        : Promise.resolve(jokes))
        .then(arr => {
            const i = Math.round(Math.random() * arr.length);
            return arr[i];
        })
}
module.exports = getJoke;



## advanced http tools 



### TODO multicrawl with selenium


#### the code

multi crawl?


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var TIMEOUT = 1000;
var CONNECTIONS = 3;

// recursively dequeue tasks
function deQueue(inputQueue, searchCallback, ctx) {
    const results = [];
    console.log(ctx.client.requestHandler.sessionID);
    const callback = typeof searchCallback === 'function'
        ? searchCallback
        : importer.import(searchCallback, Object.assign({useCache: false}, ctx));
    if(inputQueue.length > 0) {
        const item = inputQueue.shift();
        return new Promise(resolve => setTimeout(() => resolve(), 100))
            .then(() => callback(...[item, ctx]))
            .catch(e => {
                console.log(e + '');
                if((e + '').indexOf('Already') > -1 || (e + '').indexOf('session') > -1) {
                    inputQueue.push(item);
                    throw new Error('Abandoning session :(', e);
                }
            })
            .then(r => results.push(r))
            .then(() => deQueue(inputQueue, searchCallback, ctx))
            .then(r => results.concat(r))
            .catch(e => console.log(e))
    } else {
        return results;
    }
}

// create a number of individual selenium sessions and dequeue the tasks with the callback search
function multiCrawl(inputList, searchCallback) {
    var indexes = Array.from(Array(Math.min(inputList.length, CONNECTIONS)).keys());
    var connections = [];
    var promises = indexes.map((s, i) => resolve => {
        const client = runSeleniumCell(false, false);
        return client
            // skip this if error
            //.then(() => connections[i].onlyOneWindow())
            //.then(() => connections[i].resizeWindow())
            .then(ctx => {
                connections.push(ctx);
                resolve(ctx)
            })
            .catch(e => {
                console.log(e);
                resolve(null);
            })
    });
    var queue = [].concat(inputList);
    var count = 0;
    return importer.runAllPromises(promises)
        .then(() => {
            console.log(connections.map(c => c.client.requestHandler.sessionID));
        })
        .then(() => {
            return connections[0].client
                .scanning(true)
                .then(() => connections[0].getAllSessionUrls())
                .scanning(false)
        })
        .then(() => console.log('done loading sessions'))
        .then(() => Promise.all(connections.map(ctx => deQueue(queue, searchCallback, ctx))))
        .then(r => [].concat([], ...r))
}
module.exports = multiCrawl;



### crawl domain?



#### the code



In [35]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {
    cacheFilename,
    doBrowserRequest
} = importer.import('domain crawler tools')

async function crawlAll(url, depth, searches) {
    if(!depth) depth = 3 // TODO: minutes depth using time range?
    if(!searches) searches = []
    
    var requests = (typeof url === 'string' ? [url] : url )
        .filter((l, i, arr) => arr.indexOf(l) === i)
        .filter(l => !l.includes('data:'))
    var searches2 = []
    for(var i = 0; i < requests.length; i++) {
        var l = requests[i]
        var result = await doBrowserRequest(l)
        searches2.push(result)
    }
    var links = [].concat.apply([], searches2.map(s => !s ? [] : s.styles.concat(s.links)))
    var crawl = depth > 1
        ? await crawlAll(links,
                         depth - 1, searches.concat(searches2))
        : searches.concat(searches2)
    
    if(depth === 1) {
        var filePath = cacheFilename(crawl[0].url)
        fs.writeFileSync(filePath, JSON.stringify(crawl, null, 4))
    }
    //return crawl
}

module.exports = crawlAll
//var importer = require('../Core')
//var crawlAll = importer.import('crawl domain')

if(typeof $$ !== 'undefined') {
    $$.async()
    crawlAll('https://google.com', 2)
        .then(r => $$.sendResult('done'))
        .catch(e => $$.sendError(e))
}



importing domain crawler tools - 1 cell - data collection.ipynb[6]
/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[6] aka domain crawler tools
compiling /Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[6] aka domain crawler tools with module,importer,exports,__dirname,__filename,$$
importing glob files - 1 cell - files.ipynb[2]
importing select tree - 1 cell - patterns.ipynb[2]
https://www.google.com/
https://www.google.com/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png
https://www.google.com/xjs/_/js/k=xjs.s.en_US.R8dX-hLA4nk.O/ck=xjs.s.8N3EJVB8puY.L.W.O/am=AAAAgEUAu-4AIP9vBYAAgB0DAAABYcEGC4QhocJYnYAIEA/d=1/exm=Fkg7bd,HcFEGb,IvlUe,MC8mtf,OF7gzc,RMhBfe,T4BAC,TJw5qb,TbaHGc,Y33vzc,cdos,cr,csi,d,hsm,iDPoPb,jsa,mvYTse,tg8oTe,uz938c,vWNDde,ws9Tlc,yQ43ff/ed=1/dg=2/ct=zgms/rs=ACT90oHR0bkKQ-1_CyQGavk4eU7R5R4yaA/m=aa,abd,async,dvl,fEVMic,foot,ifl,lu,m,mUpTid,mu,sb_wiz,sf,spch,wft,xz7cCd?xjs=s1
https://apis.google.com/_/scs/abc-static/_/js/k=gapi.gap

'done'

#### domain crawler tools?


In [30]:
var {URL, pathToFileURL} = require('url')
var fs = require('fs')
var path = require('path')
var puppeteer = require('puppeteer')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var {selectDom} = importer.import('select tree')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function cacheFilename(url) {
    if(typeof url === 'string')
        url = new URL(url)
    const time = new Date()
    var file = url.hostname.replace(/[^a-z0-9-_]/ig, '_')
       + '-' + time.getFullYear()
       + '-' + (time.getMonth() + 1)
       + '-' + time.getDate() + '.json'
    return path.join(project, file)
}

function findCache(host) {
    const crawl = glob(
        '**/*' + host.replace(/[^a-z0-9_-]/ig, '_') + '*',
        project
    )
    crawl.sort((a, b) => {
        return fs.statSync(b).mtime.getTime() - fs.statSync(a).mtime.getTime()
    })
    return crawl
}

// TODO: make a utility for this because it keeps coming up here and in convert spreadsheet
function getAllLinks(url, doc) {
    var r = selectDom({
        links: ['//a/@href|(//img|//iframe|//audio)[@src]/@src'],
        styles: ['//link/@href|//style/@text()'],
        html: ['/*']
    }, doc)
    return {
        links: r.links.map(s => new URL(s, url).href),
        styles: r.styles.map(s => new URL(s, url).href),
        html: r.html.map(h => h.outerHTML).join('')
    }
}

var browser
async function doBrowserRequest(url) {
    
    if(!browser) browser = await puppeteer.launch()
    const page = await browser.newPage()
    var response = await page.goto(url)
    await new Promise(resolve => setTimeout(resolve, 1000))
    console.log(url)
    var type = response.headers()['content-type'].split(';')[0]
    var buffer = await response.buffer()
    var url = await page.url()

    var result = getAllLinks(url, await page.content())
    result.type = type
    result.url = url
    result.content = type.includes('text/')
        ? buffer.toString()
        : `data:${type};base64,${buffer.toString('base64')}`
    
    await page.close()
    //await browser.close()
    //browser = null
    return result
}
    
module.exports = {
    cacheFilename,
    findCache,
    getAllLinks,
    doBrowserRequest
}
    

importing glob files - 1 cell - files.ipynb[2]
importing select tree - 1 cell - patterns.ipynb[2]


{
  cacheFilename: [Function: cacheFilename],
  findCache: [Function: findCache],
  getAllLinks: [Function: getAllLinks],
  doBrowserRequest: [AsyncFunction: doBrowserRequest]
}

#### schedule crawl domain?



In [ ]:
var google = require('googleapis');
var importer = require('../Core');
var {
    getOauthClient,
    ISODateString,
    createNewEvent,
} = importer.import([
    'convert date iso',
    'create new calendar event',
    'import google calendar api',
]);

var options = {
    calendarId: 'aws'
}

function scheduleSearch(search) {
    const parameters = {
        query: search
    }
    const newDate = new Date();
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => createNewEvent('crawl domain', JSON.stringify(parameters, null, 4), options))
}
module.exports = scheduleSearch;


## html tools

TODO: move this to Languages/html.ipynb.



### find search boxes on a page



#### the code

search results as json?

In [ ]:
function searchResultsToJson(url) {
    console.log(url + ' - ' + client.requestHandler.sessionID);
    return client
        .url(url)
        .pause(2000)
        .getAllXPath({
            query: '//input[contains(@aria-label, "Search")]/@value'
            +
            '|//input[contains(@aria-label, "search")]/@value'
            +
            // yahoo
            '|//label[contains(., "Search")]/following::*//input[@type="text"]/@value' 
            +
            '|//input[contains(@class, "Search")]/@value'
            +
            // wolfram
            '|//input[contains(@name, "query")]/@value'
            +
            // duckduckgo
            '|//input[contains(@id, "search")]/@value'
            +
            // yandex
            '|//input[contains(@aria-label, "Request")]/@value',
            results: [
                '//h3|//h2|div[contains(@class, "title")]'
                +
                // ask
                '|//*[contains(@class, "item-title")]',
                {
                    name: './/text()',
                    summary: './/following-sibling::div//text()'
                }
            ]
        })
        .then(r => {
            return {
                url: url,
                query: typeof r.query === 'string'
                    ? r.query
                    : r.query[0],
                results: r.results.map(s => ({
                    name: typeof s.name === 'string'
                        ? s.name : s.name.join('\n'),
                    summary: typeof s.summary === 'string'
                        ? s.summary : s.summary.join('\n')
                }))
            };
        })
        .catch(e => {
            console.log(e)
            return {
                url: url,
                query: null,
                results: []
            }
        })
}
module.exports = searchResultsToJson;
